<a href="https://colab.research.google.com/github/nassim1014/Fine_tuning_LoRA/blob/main/LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: i want to use the huggingface transformers library , what is the first step ?

!pip install transformers
import transformers
transformers.__version__


'4.50.2'

In [ ]:
# prompt: i want to use the huggingface peft library , torch and numpy . what is the first step

!pip install peft
!pip install torch
!pip install numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
from datasets import load_dataset, DatasetDict , Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model , LoraConfig
import evaluate
import torch
import numpy as np

In [ ]:
model_checkpoint = 'distilbert-base-uncased'
#label maps sentiment analysis
id2label = {0: 'NEGATIVE', 1: 'NEUTRAL', 2: 'POSITIVE'}
label2id = {'NEGATIVE': 0, 'NEUTRAL': 1, 'POSITIVE': 2}

#generate calssification model from model_checkpoint

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels = 3,
    id2label = id2label,
    label2id = label2id
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset = load_dataset("shawhin/imdb-truncated")
dataset

README.md:   0%|          | 0.00/592 [00:00<?, ?B/s]

(…)-00000-of-00001-5a744bf76a1d84b2.parquet:   0%|          | 0.00/836k [00:00<?, ?B/s]

(…)-00000-of-00001-a3a52fabb70c739f.parquet:   0%|          | 0.00/853k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

def tokenize_function(examples):
  text = examples['text']

  tokenizer.truncation_side = "left"
  tokenizer_inputs = tokenizer(
      text,
      max_length = 512,
      truncation = True,
      return_tensors ="np"
  )
  return tokenizer_inputs

#add pad token if none exists
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token':'[PAD]'})
  model.resize_token_embeddings(len(tokenizer))

#tokenize training and validation data
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
  predictions , labels = p
  predictions = np.argmax(predictions, axis = 1)
  return {"accuracy":accuracy.compute(predictions = predictions,
                                      references = labels)}

In [ ]:
# prompt: generate a list of 5 strings, that represent movie reviews. they will be used for sentiment analysis

reviews = [
    "This movie was absolutely fantastic! The acting was superb, the plot was engaging, and the visuals were stunning.",
    "I was thoroughly disappointed. The storyline was predictable, the acting was mediocre, and the pacing was slow.",
    "A decent watch, nothing spectacular. It had its moments, but overall it wasn't memorable.",
    "This is one of the worst movies I've ever seen. The plot was nonsensical, the acting was terrible, and it was just a waste of time.",
    "I really enjoyed this film. It was a heartwarming story with great characters and performances."
]


## untrained movie predictions

In [ ]:
for text in reviews :
  inputs = tokenizer.encode(text, return_tensors='pt')
  logits = model(inputs).logits
  predictions = torch.argmax(logits)

  print(text + " - " + id2label[predictions.tolist()])

This movie was absolutely fantastic! The acting was superb, the plot was engaging, and the visuals were stunning. - NEGATIVE
I was thoroughly disappointed. The storyline was predictable, the acting was mediocre, and the pacing was slow. - NEGATIVE
A decent watch, nothing spectacular. It had its moments, but overall it wasn't memorable. - NEGATIVE
This is one of the worst movies I've ever seen. The plot was nonsensical, the acting was terrible, and it was just a waste of time. - NEGATIVE
I really enjoyed this film. It was a heartwarming story with great characters and performances. - NEGATIVE


## Fine Tuning

In [ ]:
import torch

# Vérifier si le GPU est disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [ ]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                         r = 4,
                         lora_alpha = 32,
                         lora_dropout = 0.01,
                         target_modules = ["q_lin"])

In [ ]:
#get ready to fintuned model
model = get_peft_model(model , peft_config)
model.print_trainable_parameters()
model = model.to(device)

trainable params: 629,763 || all params: 67,585,542 || trainable%: 0.9318


In [ ]:
#hyperparam
lr = 1e-2
batch_size = 10
num_epochs = 10

#training
training_args = TrainingArguments(
    output_dir = model_checkpoint + "-lora-text_classification",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

<ipython-input-37-43ab8b28402d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.824289,{'accuracy': 0.506}
2,No log,0.851402,{'accuracy': 0.494}
3,No log,0.725714,{'accuracy': 0.489}
4,No log,0.722363,{'accuracy': 0.525}
5,0.766100,0.723758,{'accuracy': 0.512}
6,0.766100,0.704899,{'accuracy': 0.522}
7,0.766100,0.711286,{'accuracy': 0.485}
8,0.766100,0.693335,{'accuracy': 0.524}
9,0.766100,0.696641,{'accuracy': 0.518}
10,0.708600,0.693583,{'accuracy': 0.542}


Trainer is attempting to log a value of "{'accuracy': 0.506}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.494}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.489}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.525}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.512}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This i

TrainOutput(global_step=1000, training_loss=0.7373522033691406, metrics={'train_runtime': 482.7173, 'train_samples_per_second': 20.716, 'train_steps_per_second': 2.072, 'total_flos': 1286919437506200.0, 'train_loss': 0.7373522033691406, 'epoch': 10.0})

In [ ]:
for text in reviews :
  inputs = tokenizer.encode(text, return_tensors='pt').to(device)
  with torch.no_grad():
        outputs = model(inputs)
        logits = outputs.logits

  predictions = torch.argmax(logits)

  print(text + " - " + id2label[predictions.tolist()])

This movie was absolutely fantastic! The acting was superb, the plot was engaging, and the visuals were stunning. - NEGATIVE
I was thoroughly disappointed. The storyline was predictable, the acting was mediocre, and the pacing was slow. - NEGATIVE
A decent watch, nothing spectacular. It had its moments, but overall it wasn't memorable. - NEGATIVE
This is one of the worst movies I've ever seen. The plot was nonsensical, the acting was terrible, and it was just a waste of time. - NEGATIVE
I really enjoyed this film. It was a heartwarming story with great characters and performances. - NEUTRAL
